# ParametrizationDefinition

### settings.json

In order to estimate multimodal features of neurophysiological data, certain parametrization steps are required. 
Here the following two parametrization files are explained: 
 - `settings.json`
 - `df_M1.csv`
 
The `settings.json` specifies the general feature processing. Most importantly, all provided features can be enabled and disabled: 
```json
"methods": 
{
        "raw_resampling": false,
        "normalization": true,
        "kalman_filter": true,
        "re_referencing": true,
        "notch_filter": true,
        "bandpass_filter": true,
        "raw_hjorth": true,
        "sharpwave_analysis": true,
        "return_raw": true,
        "project_cortex": false,
        "project_subcortex": false,
        "pdc": false,
        "dtf": false
}
```

**raw_resampling** defines a resampling rate to which the original data is downsampled to. This can be of advantage, since high sampling frequencies automatically require higher computational cost. In the method specific settings the resampling frequency can be defined: 

```json
"raw_resampling_settings": {
        "resample_freq": 1000
    }
```

**normalization** allows for normalizing the past *normalization_time* according to either the *mean* or *median* *normalization_method* in the following manner for *median*:

$X_{norm} = \frac{X - median(X)}{median(X)}$
```json
"raw_normalization_settings": {
        "normalization_time": 10,
        "normalization_method": "median"
    }

```

**kalman_filtering** is motivated by filtering estimated band power features using the white noise acceleration model (see ["Improved detection of Parkinsonian resting tremor with feature engineering and Kalman filtering"](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6927801/) Yao et al 19) for a great reference. The white noise acceleration model get's specified by the $T_p$ prediction interval (Hz), and the process noise is then defined by $\sigma_w$ and $\sigma_v$: 

$
  Q=
  \left[ {\begin{array}{cc}
   \sigma_w^2\frac{T_p^{3}}{3} & \sigma_w^2\frac{T_p^2}{2}\\
   \sigma_w^2\frac{T_p^2}{3} & \sigma_w^2T_p\\
  \end{array} } \right]
$


```json
"kalman_filter_settings": {
        "Tp": 0.1,
        "sigma_w": 0.7,
        "sigma_v": 1,
        "frequency_bands": [
            "low gamma",
            "high gamma",
            "all gamma"
        ]
    }
```
Individual frequency bands (specified in the **bandpass_filter_settings**) can be selected for Kalman Filtering (see ["Real-time epileptic seizure prediction using AR models and support vector machines"](https://pubmed.ncbi.nlm.nih.gov/20172805/) (Chisci et al 10)) 

**re_referencing** constitutes an important aspect of electrophysiological signal processing. Most commonly bipolar and common average rereferencing are applied. Due to that reason a seperate file, `df_M1.csv` can be specified before feature estimation, or it is automatically setup during parametrization.`df_M1.csv` contains a column *rereference*, specifying the rereference methods 
 - *average* for common average rereference (across a channel type, e.g. ecog)
 - bipolar rereferencing, by specifying the channel name to rereference to, e.g. *LFP_BS_STN_L_1*
 - *none* for no rereferencing being used for this particular channel 

**notch_filter** enables notch filters at the specified line_noise and it's harmonics. 

**bandpass_filter** enables band power feature estimation. Settings are defined in such manner: 
```json
"bandpass_filter_settings": {
    "frequency_ranges": {
        "theta": [
            [
                4,
                8
            ],
            1000
            ]
    }, 
    "bandpower_features": {
        "activity": true,
        "mobility": true,
        "complexity": true
    }
}
```

Here the name (in this case *theta*) get's specified, and in the the subsequent list the frequency range (here from 4 - 8 Hz) is defined. Then, the last parameter defines a time range in which FIR filtered data is used for feature estimation in milliseconds. Here the previous 1000 ms are used to estimate features based on the FIR filtered signals in the range of 4 to 8 Hz. This might be beneficial when using shorter frequency bands, e.g. gamma, where estimating band power in a range of e.g. 100 ms might result in a temporal more specified feature calculation. 
A common way to estimate band power is to take the variance of FIR filtered data. This is equavilent to the activity [Hjorth](https://en.wikipedia.org/wiki/Hjorth_parameters) parameter. The last key in the *bandpass_filter_settings* allows to take the *activity*, *mobility* and *complexity* Hjorth parameters as well. For estimating Hjorth parameters of the raw unfiltered signal, the **raw_hjorth** method can be enabled. 

**sharpwave_analysis** allows for calculation of temporal sharpwave features. See ["Brain Oscillations and the Importance of Waveform Shape"](https://www.sciencedirect.com/science/article/abs/pii/S1364661316302182) Cole et al 17 for a great motivation to use these features. Here, sharpwave features are estimated using a prior bandpass filter  between *filter_low_cutoff* and *filter_high_cutoff*. The sharpwave peak and trough features can be calculated, defined by the *estimate* key. According to a current data batch one or more sharpwaves can be detected. The subsequent feature is returned rather by the *mean, median, maximum, minimum or variance* as defined by the *estimator*. 
```json
"sharpwave_analysis_settings": {
    "sharpwave_features": {
        "peak_left": true,
        "peak_right": true,
        "trough": true,
        "width": true,
        "prominence": true,
        "interval": true,
        "decay_time": true,
        "rise_time": true,
        "sharpness": true,
        "rise_steepness": true,
        "decay_steepness": true,
        "slope_ratio": true
    },
    "filter_low_cutoff": 5,
    "filter_high_cutoff": 90,
    "detect_troughs": {
        "estimate": true,
        "distance_troughs": 5,
        "distance_peaks": 1
    },
    "detect_peaks": {
        "estimate": true,
        "distance_troughs": 1,
        "distance_peaks": 5
    },
    "estimator": {
        "mean": true,
        "median": true,
        "max": true,
        "min": true,
        "var": true
    }
}
```
A separate tutorial on sharpwave features is provided in the documentation. 

Next, raw signals can be returned, specifed by the **return_raw** method. 

**projection_cortex** and **projection_subcortex** allows then feature projection of individual channels to a common subcortical or cortical grid, defined by *grid_cortex.tsv* and *subgrid_cortex.tsv*. For both projections a *max_dist* parameter needs to be specified, in which data is linearly interpolated, weighted by their inverse grid point distance. 

Additionally **pdc** and **dtf** enable partical directed coherence and direct transfer function, to enable connectiviy features for certain *frequency_bands* between specific channels. 

### df_M1

As described above, rereferencing will be estimated automatically given the specified channel types. To demonstrate a typical rereferencing scheme, here the example BIDS data in 'py_neuromodulation/tree/main/pyneuromodulation/tests/data' is read.

First, the path needs to specified and necessary libaries imported:

In [1]:
import sys
from bids import BIDSLayout
import os
import json
from pathlib import Path

# first parent to get example folder, second py_neuromodulation folder
PATH_PYNEUROMODULATION = Path("__file__").absolute().parent.parent  # "__file__" required for ipynb
sys.path.append(os.path.join(PATH_PYNEUROMODULATION, 'pyneuromodulation'))
sys.path.append(os.path.join(Path("__file__").absolute().parent.parent,'examples'))

import start_BIDS
import settings as nm_settings
import IO

In [2]:
print("current working directory: " + os.getcwd())
print("PATH_PYNEUROMODULATION: " + str(PATH_PYNEUROMODULATION))
BIDS_EXAMPLE_PATH = os.path.abspath(os.path.join(PATH_PYNEUROMODULATION, 'pyneuromodulation',
                                    'tests', 'data'))

# write BIDS example path in settings.json
with open(os.path.join(os.pardir, 'examples', 'settings.json'), encoding='utf-8') as json_file:
    settings = json.load(json_file)
settings["BIDS_path"] = BIDS_EXAMPLE_PATH

# write relative feature output folder
settings["out_path"] = os.path.abspath(os.path.join(PATH_PYNEUROMODULATION, 'pyneuromodulation',
                                    'tests', 'data', 'derivatives'))
with open(os.path.abspath(os.path.join(os.pardir, 'examples', 'settings.json')), 'w') as f:
    json.dump(settings, f, indent=4)

PATH_RUN = os.path.join(BIDS_EXAMPLE_PATH, 'sub-testsub', 'ses-EphysMedOff',
                        'ieeg', "sub-testsub_ses-EphysMedOff_task-buttonpress_ieeg.vhdr")


current working directory: C:\Users\ICN_admin\Documents\py_neuromodulation\sphinx
PATH_PYNEUROMODULATION: C:\Users\ICN_admin\Documents\py_neuromodulation


Then the df_M1.csv file is automatically specifed given the respective channel types. Since one channel contains an 'analog' substring, it is automaticaly assginged to be a *target* channel. 

In [3]:
os.chdir(os.path.join(os.pardir,'pyneuromodulation'))
sys.path.append(os.path.join(os.pardir,'examples'))

# read and test settings first to obtain BIDS path
settings_wrapper = nm_settings.SettingsWrapper(settings_path=os.path.join(os.pardir, 'examples', 'settings.json'))

# read BIDS data
raw_arr, raw_arr_data, fs, line_noise = IO.read_BIDS_data(PATH_RUN, settings_wrapper.settings["BIDS_path"])

# read df_M1 / create M1 if None specified
settings_wrapper.set_M1(m1_path=None, ch_names=raw_arr.ch_names,
                        ch_types=raw_arr.get_channel_types())
settings_wrapper.set_fs_line_noise(fs, line_noise)

Reading settings.json.
Testing settings.
No Error occurred when testing the settings.
Extracting parameters from C:\Users\ICN_admin\Documents\py_neuromodulation\pyneuromodulation\tests\data\sub-testsub\ses-EphysMedOff\ieeg\sub-testsub_ses-EphysMedOff_task-buttonpress_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\Documents\py_neuromodulation\pyneuromodulation\tests\data\sub-testsub\ses-EphysMedOff\ieeg\sub-testsub_ses-EphysMedOff_task-buttonpress_channels.tsv.
Reading in coordinate system frame Other: None.
Reading electrode coords from C:\Users\ICN_admin\Documents\py_neuromodulation\pyneuromodulation\tests\data\sub-testsub\ses-EphysMedOff\ieeg\sub-testsub_ses-EphysMedOff_acq-StimOff_space-mni_electrodes.tsv.
The read in electrodes file is: 
 [('name', ['ECOG_AT_SM_L_1', 'ECOG_AT_SM_L_2', 'ECOG_AT_SM_L_3', 'ECOG_AT_SM_L_4', 'ECOG_AT_SM_L_5', 'ECOG_AT_SM_L_6', 'LFP_STN_R_234', 'LFP_STN_R_567', 'LFP_BS_STN_L_1', 'LFP_STN_L_234', 'LFP_STN_L_567

C:\Users\ICN_admin\Documents\py_neuromodulation\pyneuromodulation\IO.py:32: RuntimeWarning: Did not find any events.tsv associated with sub-testsub_ses-EphysMedOff_task-buttonpress.

The search_str was "C:\Users\ICN_admin\Documents\py_neuromodulation\pyneuromodulation\tests\data\sub-testsub\**\sub-testsub_ses-EphysMedOff*events.tsv"
  raw_arr = mne_bids.read_raw_bids(bids_path)
C:\Users\ICN_admin\Documents\py_neuromodulation\pyneuromodulation\IO.py:32: RuntimeWarning: Defaulting coordinate frame to unknown from coordinate system input Other
  raw_arr = mne_bids.read_raw_bids(bids_path)
C:\Users\ICN_admin\Documents\py_neuromodulation\pyneuromodulation\IO.py:32: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['ANALOG_ROT_R_1']
  raw_arr = mne_bids.read_raw_bids(bids_path)
C:\Users\ICN_admin\Documents\py_neuromodulation\pyneuromodulation\IO.py:32: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw

Based on 'seeg' type (could be also 'dbs'), the subsequent depth local field potential channel is selected for rereferencing. For 'ecog' channels, the average rereference function is automatically assigned. 

Note here also that channels can be 'used' for feature analysis.

In [4]:
settings_wrapper.df_M1

,name,rereference,used,target,type,status,new_name
0,LFP_STN_R_234,LFP_STN_R_567,1,0,seeg,good,LFP_STN_R_234-LFP_STN_R_567
1,LFP_STN_R_567,LFP_STN_R_234,1,0,seeg,good,LFP_STN_R_567-LFP_STN_R_234
2,LFP_BS_STN_L_1,LFP_STN_L_567,1,0,seeg,good,LFP_BS_STN_L_1-LFP_STN_L_567
3,LFP_STN_L_234,LFP_BS_STN_L_1,1,0,seeg,good,LFP_STN_L_234-LFP_BS_STN_L_1
4,LFP_STN_L_567,LFP_STN_L_234,1,0,seeg,good,LFP_STN_L_567-LFP_STN_L_234
5,ECOG_AT_SM_L_1,average,1,0,ecog,good,ECOG_AT_SM_L_1-avgref
6,ECOG_AT_SM_L_2,average,1,0,ecog,good,ECOG_AT_SM_L_2-avgref
7,ECOG_AT_SM_L_3,average,1,0,ecog,good,ECOG_AT_SM_L_3-avgref
8,ECOG_AT_SM_L_4,average,1,0,ecog,good,ECOG_AT_SM_L_4-avgref
9,ECOG_AT_SM_L_5,average,1,0,ecog,good,ECOG_AT_SM_L_5-avgref
